### Start and Run All

In [2]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
# from datetime import date, timedelta
from datetime import datetime
from time import sleep, strftime

# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-25 20:02:32


In [3]:
url = "https://www.set.or.th/en/market/product/stock/quote/"
percent_span = '' 

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"Obsidian path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
Obsidian path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
file_name = "name-ttl.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file : {input_file}") 

Input file : C:\Users\PC1\OneDrive\A5\Data\name-ttl.csv


In [7]:
file_name = "price-hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [8]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\price-hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\price-hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\price-hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\price-hilo.csv


In [9]:
def cvt_str_to_flt(input_str):
    # Check for the special pattern and return 0
    if input_str == '(-)':
        return 0
    
    # Remove parentheses and percent sign
    cleaned_str = input_str.replace('(', '').replace(')', '').replace('%', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle the case where the conversion fails
        return None
 

In [10]:
def get_stock_data(in_df):

    in_df.columns = ['name']
    
    output_columns = ['name', 'price', 'today_low', 'today_high', 'year_low', 'year_high', 'percent', 'updated_at']
    # Initialize an empty DataFrame with specified columns
    out_df = pd.DataFrame(columns=output_columns)
    
    for index, row in in_df.iterrows():
        name = str(row['name']).upper()
        response = request.urlopen(url + name + '/price')
        html_page = BeautifulSoup(response, 'html.parser')
        
  #      price = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info').text.strip()        
  #      if (price == '-'):
  #          price = '0.00'        

        price_div = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info')
        if price_div:
            price = price_div.text.strip()
        else:
            print(f"Price div not found for {name}")
            price = '0.00'     
            
        # Try to find the <h3> tag with the class 'theme-success'
        h3_tag = html_page.find('h3', class_='theme-success')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-danger')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)    
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-normal')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)   
                
        percent = cvt_str_to_flt(percent_span)
                
        values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
        today_low = values_low[0].text
        if (today_low == '-'):
            today_low = '0.00'
        
        values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
        today_high = values_high[0].text
        if (today_high == '-'):
            today_high = '0.00'         
        
        year_low = values_low[1].text        
        year_high = values_high[1].text     
        
        # Find the div with the specific class, then find the span within it
        div = html_page.find('div', class_='price-detail-date d-flex align-items-center text-nowrap fs-12px')
        if div:
            span = div.find('span')
            if span:
                text = span.get_text()
#                 print(text)
            else:
                print("Span not found")
        else:
            print("Div not found")
            
        # Extract the date part from the string
        # Assuming the date is always in the format 'dd Mon yyyy'
        date_str = ' '.join(text.split()[2:5])

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%d %b %Y')

        # Format the datetime object to 'yyyy-mm-dd'
        updated_at = date_obj.strftime('%Y-%m-%d')

#         print(updated_at)
        
        # Append row to the DataFrame and print the row
        out_df.loc[len(out_df)] = [name, price, today_low, today_high, year_low, year_high, percent, updated_at]
#       print(out_df.iloc[-1].to_string(header=False, index=False)) 
        print(','.join(map(str, out_df.iloc[-1])))
        sleep(1)
        
    return out_df  # return the final dataframe with all values populated

In [11]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  08:02 PM


In [12]:
df = pd.read_csv(input_file, header=None)
out_df = get_stock_data(df)

ACE,1.30,1.28,1.30,1.23,1.49,0.78,2025-08-25
ADVANC,291.00,290.00,296.00,242.00,309.00,0.0,2025-08-25
AEONTS,107.50,104.00,109.00,90.50,148.00,3.86,2025-08-25
AH,15.00,14.70,15.00,10.00,23.20,2.74,2025-08-25
AIE,1.15,1.12,1.17,0.72,1.80,0.88,2025-08-25
AIMIRT,9.70,9.55,9.70,9.20,11.20,1.57,2025-08-25
AIT,4.92,4.88,4.96,3.86,5.30,0.41,2025-08-25
AJ,2.28,2.26,2.32,1.94,5.80,0.0,2025-08-25
AMATA,16.90,16.60,17.00,11.20,31.00,1.2,2025-08-25
ANAN,0.50,0.49,0.51,0.37,0.85,0.0,2025-08-25
AOT,38.00,37.50,38.75,26.75,65.00,1.33,2025-08-25
AP,7.60,7.45,7.60,5.60,10.50,2.7,2025-08-25
ASIAN,7.75,7.65,7.80,6.50,10.00,0.65,2025-08-25
ASK,7.50,7.50,7.60,6.20,15.30,0.0,2025-08-25
ASP,2.46,2.40,2.46,1.81,2.70,0.82,2025-08-25
ASW,6.25,6.15,6.25,5.55,8.20,0.81,2025-08-25
AWC,2.40,2.38,2.46,1.55,3.94,-0.83,2025-08-25
BA,14.00,14.00,14.40,10.60,26.00,0.0,2025-08-25
BAM,8.20,8.00,8.30,5.35,10.20,3.14,2025-08-25
BANPU,4.82,4.82,4.92,4.00,7.50,-0.41,2025-08-25
BAY,23.60,23.30,23.60,20.70,29.00,0.85,2025-08-25

In [13]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  08:07 PM


In [14]:
out_df.query('price == "-"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
205,TYCN,-,0.00,0.00,1.62,2.62,0.0,2025-08-25


In [15]:
out_df.shape

(216, 8)

In [16]:
out_df = out_df[out_df['name'] != 'BCT']

In [17]:
out_df = out_df[out_df['name'] != 'LPF']

In [18]:
out_df = out_df[out_df['name'] != 'TYCN']

In [19]:
out_df = out_df[out_df['name'] != 'GRAMMY']

In [20]:
out_df = out_df[out_df['name'] != 'MST']

In [21]:
out_df = out_df[out_df['name'] != 'SPC']

In [22]:
out_df = out_df[out_df['name'] != 'SPRIME']

In [23]:
out_df = out_df[out_df['name'] != 'FPT']

In [24]:
out_df = out_df[out_df['name'] != 'GGC']

In [25]:
out_df.shape

(208, 8)

In [26]:
out_df['price'] = out_df['price'].astype('float')
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')
out_df['percent'] = out_df['percent'].astype('float')

In [27]:
sorted_out_df = out_df.sort_values('percent')

In [28]:
sorted_out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
40,BTSGIF,2.5,2.46,2.5,2.54,3.52,-6.02,2025-08-25
171,STANLY,180.0,176.00,181.5,180.00,225.00,-0.28,2025-08-25


In [29]:
sorted_out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
24,BCPG,8.5,8.35,8.95,5.05,8.75,4.29,2025-08-25


In [30]:
out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
24,BCPG,8.5,8.35,8.95,5.05,8.75,4.29,2025-08-25


In [31]:
file_name = "Price-Hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [32]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\Price-Hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-Hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-Hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-Hilo.csv


In [33]:
out_df.to_csv(output_file, index=False)  
out_df.to_csv(god_file, index=False) 
out_df.to_csv(icd_file, index=False)  
out_df.to_csv(osd_file, index=False) 

In [34]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-25 20:07:39
